In [1]:
import os
import re
from string import punctuation
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anmrt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anmrt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^а-яё\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [ ]:
#ПРОБЛЕМА С КОДИРОВКОЙ У ПОСЛЕДНИХ ТРЕХ ФАЙЛОВ

In [ ]:
# corpus = []

# with open('HP1RUS.txt', 'r', encoding='utf-8') as file:
#     lines = file.readlines()[7:]
# text = ' '.join(lines)
# processed_text = preprocess_text(text)
# sentences = nltk.sent_tokenize(processed_text, language='russian')
# corpus.extend(sentences)

# with open('HP2RUS.txt', 'r', encoding='utf-8') as file:
#     lines = file.readlines()[7:]
# text = ' '.join(lines)
# processed_text = preprocess_text(text)
# sentences = nltk.sent_tokenize(processed_text, language='russian')
# corpus.extend(sentences)

# with open('HP3RUS.txt', 'r', encoding='utf-8') as file:
#     lines = file.readlines()[6:]
# text = ' '.join(lines)
# processed_text = preprocess_text(text)
# sentences = nltk.sent_tokenize(processed_text, language='russian')
# corpus.extend(sentences)

# with open('HP4RUS.txt', 'r', encoding='utf-8') as file:
#     lines = file.readlines()[3:]
# text = ' '.join(lines)
# processed_text = preprocess_text(text)
# sentences = nltk.sent_tokenize(processed_text, language='russian')
# corpus.extend(sentences)

In [7]:
def tokenize_corpus(corpus):
    tokenized_corpus = []
    
    for sentence in corpus:
        tokens = word_tokenize(sentence, language='russian')
        
        # Удалить стоп-слова и короткие слова
        stop_words = set(stopwords.words('russian'))
        filtered_tokens = [
            token for token in tokens 
            if token not in stop_words and len(token) > 2
        ]
        
        if filtered_tokens:  # Добавить только непустые списки
            tokenized_corpus.append(filtered_tokens)
    
    return tokenized_corpus

In [8]:
def save_processed_data(tokenized_corpus, filename='processed_corpus.txt'):
    with open(filename, 'w', encoding='utf-8') as f:
        for tokens in tokenized_corpus:
            f.write(' '.join(tokens) + '\n')
    print("сохранено")
    print(filename)

In [11]:
with open('HP1RUS.txt', 'r', encoding='utf-8') as infile, \
     open('Data.txt', 'w', encoding='utf-8') as outfile:
    lines = infile.readlines()
    outfile.writelines(lines[7:])

In [12]:
with open('Data.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [13]:
processed_text = preprocess_text(text)
sentences = nltk.sent_tokenize(processed_text, language='russian')

In [ ]:
print(len(sentences)) #вывод 1, поскольку это все 1 строка в обработанном тексте

1


In [15]:
print(sentences)

['мистер и миссис дарсли из дома номер четыре по бирючинному проезду могли бы с гордостью сказать что они слава богу совершенно нормальные люди они были бы последними от кого вы могли бы ожидать участия в чемнибудь странном и таинственном потому что они совершенно не одобряли подобной чепухи мистер дарсли работал директором фирмы граннингз которая выпускала сверла он был крупный крепкий мужчина с очень короткой шеей и очень большими усами миссис дарсли была худой и светловолосой зато ее шеи с лихвой хватило бы на двоих этим подарком природы она усердно пользовалась большую часть времени шпионя за соседями через садовые изгороди у дарсли рос сын дадли и они были уверены что лучшего мальчика не сыщешь на всем свете у дарсли было все что нужно но кроме того у них была тайна и больше всего они боялись что ктонибудь раскроет её они с ужасом представляли что случится если ктото вдруг узнает о поттерах миссис поттер приходилась миссис дарсли сестрой которую та не видела вот уже несколько лет 

In [16]:
tokenized_corpus = tokenize_corpus(sentences)

In [17]:
save_processed_data(tokenized_corpus)

сохранено
processed_corpus.txt


In [20]:
# Всего токенов
print(sum(len(tokens) for tokens in tokenized_corpus))

39978


In [21]:
pip install scikit-learn

  Using cached scikit_learn-1.7.2-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.16.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.2-cp311-cp311-win_amd64.whl (8.9 MB)
Using cached scipy-1.16.3-cp311-cp311-win_amd64.whl (38.7 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
pip install gensim

  Using cached gensim-4.4.0-cp311-cp311-win_amd64.whl.metadata (8.6 kB)
Using cached gensim-4.4.0-cp311-cp311-win_amd64.whl (24.4 MB)
   ---------------------------------------- 0.0/63.9 kB ? eta -:--:--
   ------ --------------------------------- 10.2/63.9 kB ? eta -:--:--
   ------------------- -------------------- 30.7/63.9 kB 435.7 kB/s eta 0:00:01
   ------------------- -------------------- 30.7/63.9 kB 435.7 kB/s eta 0:00:01
   -------------------------------------- - 61.4/63.9 kB 365.7 kB/s eta 0:00:01
   ---------------------------------------- 63.9/63.9 kB 287.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
   ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
   -------------------- ------------------- 30.7/60.4 kB 435.7 kB/s eta 0:00:01
   -------------------- ------------------- 30.7/60.4 kB 435.7 kB/s eta 0:00:01
   --------------------------------- ------ 51.2/60.4 kB 292.6 kB/s eta 0:00:01
   ---------------------


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np

In [24]:
model = Word2Vec(
    sentences=tokenized_corpus,
    vector_size=100,
    window=5,             # размер окна контекста
    min_count=3,          # минимальная частота слова
    sg=1,                 # алгоритм Skip-Gram (при 1) или CBOW (0)
    workers=4,            # количество ядер процессора
    epochs=20
)

In [ ]:
print(len(model.wv.key_to_index)) # Размер словаря слов

3000


In [26]:
model.save("harry_potter_rus.model")

In [28]:
def print_similar_words(model, word, topn=10):
    if word in model.wv:
        similar = model.wv.most_similar(word, topn=topn)
        print(f"\nБлижайшие слова к '{word}':")
        for i, (similar_word, similarity) in enumerate(similar, 1):
            print(f"{i:2d}. {similar_word:<15} {similarity:.4f}")

In [29]:
characters = ['гарри', 'гермиона', 'рон', 'волдеморт', 'снейп', 'мальфой']
for character in characters:
    print_similar_words(model, character)


Ближайшие слова к 'гарри':
 1. оба             0.9615
 2. купил           0.9606
 3. маггл           0.9581
 4. волшебник       0.9580
 5. тобой           0.9558
 6. внезапно        0.9552
 7. мальчишка       0.9552
 8. понял           0.9550
 9. слышал          0.9539
10. снова           0.9537

Ближайшие слова к 'гермиона':
 1. помнит          0.9950
 2. невилл          0.9949
 3. видели          0.9947
 4. которыми        0.9944
 5. зубы            0.9944
 6. привет          0.9943
 7. будут           0.9942
 8. спрашивать      0.9942
 9. слухи           0.9941
10. жабы            0.9941

Ближайшие слова к 'рон':
 1. слышал          0.9833
 2. близнецов       0.9828
 3. таким           0.9818
 4. спасибо         0.9810
 5. близнец         0.9809
 6. близнецы        0.9794
 7. место           0.9789
 8. пришел          0.9787
 9. рона            0.9778
10. плечу           0.9768

Ближайшие слова к 'волдеморт':
 1. скажет          0.9958
 2. волдеморта      0.9958
 3. ночи           

In [ ]:
def word_analogy(model, positive, negative, topn=5):
    result = model.wv.most_similar(positive=positive, negative=negative, topn=topn)
    print(f"\nАналогия: {positive} - {negative} = ?")
    for i, (word, similarity) in enumerate(result, 1):
        print(f"{i:2d}. {word:<15} {similarity:.4f}")
    return result